# API Center ❤️ all APIs

## Import from MuleSoft API Manager
![image](../../images/import-from-mulesoft.png)

Playground to experiment importing APIs from the MuleSoft Anypoint platform. We start by creating an API Center instance using [Bicep resource definition](https://learn.microsoft.com/en-us/azure/templates/microsoft.apicenter/services?pivots=deployment-language-bicep) and then we will use the [Anypoint Platform CLI](https://docs.mulesoft.com/anypoint-cli/latest/) to export the APIs and the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest) to import the APIs into API Center.

💡 Every step outlined below leverages the power of PowerShell scripts. This is designed so you can seamlessly integrate them into your automation workflows, CI/CD pipelines, webhooks, and more.

### Learning Objectives - Upon completing this lab, you should be able to:
- Deploy Azure API Center using Bicep.
- Configure the API Center metadata scheme for APIs, Environments and Deployments.
- Understand the Azure CLI commands to manage Azure API Center. [Full list of commands available here](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest).
- Create an enviroment with custom metadata properties 
- Register an API using the OpenAPI spec as the source of truth and assign API metadata properties.
- Register an API with individual commands to have full control over all the properties. 
- Search and discover the APIs registered in API Center.

### Prerequisites
- Install or update to the latest [.NET 8 SDK](https://dotnet.microsoft.com/en-us/download)
- Install or update to the latest [Visual Studio Code](https://code.visualstudio.com/)
- Install the [Polyglot Notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) from the VS Code marketplace
- Install or update to the latest [PowerShell](https://learn.microsoft.com/en-us/powershell/scripting/install/installing-powershell)
- Install or update to the latest [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli). The Azure API Center extension will automatically install the first time you run an [az apic](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest) command.
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)
- [Install and configure the MuleSoft Anypoint Platform CLI](https://docs.mulesoft.com/anypoint-cli/latest/)


### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- Adjust the APIC location parameter according your preferences and [region availability.](https://learn.microsoft.com/en-us/azure/api-center/overview#available-regions) 


In [ ]:

$deploymentName = Split-Path -Path (Get-Location) -Leaf
$resourceGroupName = "lab-$deploymentName" # change the name to match your naming style
$resourceGroupLocation = "westeurope"
$apicResourceNamePrefix = "apic"
$apicResourceSku = "free"
$apicResourceTags = @{
  "lab" = $deploymentName
}

# the following metadata will be created during the deployment
$metadata = @(
  @{
    "name" = 'technology'
    "schema" = '{"title":"technology","description":"The Mule technology used to serve the API","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'stage'
    "schema" = '{"title":"stage","description":"The API stage","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'isPublic'
    "schema" = '{"title":"isPublic","description":"Indication if its a public API","type":"boolean"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'organizationId'
    "schema" = '{"title":"organizationId","description":"Id for the organization associated with the Anypoint platform environment","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'environment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'clientId'
    "schema" = '{"title":"clientId","description":"Id for the client","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'environment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'anypointAppName'
    "schema" = '{"title":"anypointAppName","description":"App name for the deployment","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'deployment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'targetId'
    "schema" = '{"title":"targetId","description":"Target Id of the app deployment","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'deployment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'runtimeVersion'
    "schema" = '{"title":"runtimeVersion","description":"Runtime version of the app deployment","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'deployment'
        "required" = $false
      }
    )
  }

)

# the following environment(s) will be created during the Bicep deployment
$environments = @(
)

Write-Output "✅ Variables initialized ⌚ $(Get-Date -Format 'HH:mm:ss')"


### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [ ]:
$resourceGroupOutput = az group create --name $resourceGroupName --location $resourceGroupLocation

if ($LASTEXITCODE -ne 0) {
    Write-Output $resourceGroupOutput
} else {
    Write-Output "✅ Azure Resource Grpup $resourceGroupName created ⌚ $(Get-Date -Format 'HH:mm:ss')"
}

### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
$bicepParameters = @{
    "`$schema" = "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#"
    "contentVersion" = "1.0.0.0"
    "parameters" = @{
        "apicResourceNamePrefix" = @{ "value" = $apicResourceNamePrefix }
        "apicResourceTags" = @{ "value" = $apicResourceTags }
        "apicMetadataSchema" = @{ "value" = $metadata }
        "apicEnvironments" = @{ "value" = $environments }
    }
}

$bicepParametersJson = ConvertTo-Json -InputObject $bicepParameters -Depth 10
Set-Content -Path "params.json" -Value $bicepParametersJson

# Execute the Azure CLI command to create the deployment
az deployment group create --name $deploymentName --resource-group $resourceGroupName --template-file "main.bicep" --parameters "params.json"



### 3️⃣ Get the deployment outputs

We will set the `apicResourceName` variable with the value that was returned from the deployment 

In [ ]:
$deploymentOutput = az deployment group show --name $deploymentName -g $resourceGroupName --query properties.outputs.apicResourceName.value -o json
if ($LASTEXITCODE -ne 0) {
    Write-Output $deploymentOutput
} else {
    $apicResourceName = $deploymentOutput | ConvertFrom-Json
}
Write-Output "👉🏻 API Center name: $apicResourceName"

### 4️⃣ Extract MuleSoft Anypoint API artefacts and load them into API Center

The following script uses the MuleSoft Anypoint platform CLI to query Anypoint platform environments, APIs and runtime details to import the artifacts into API Center with the respective metadata.


In [ ]:
# comment one of the following lines to use the real MuleSoft data or the local file
$anypointEnvsJson = anypoint-cli-v4 account:environment:list --output=json
# $anypointEnvsJson = Get-Content -Raw 'account_environment_list.json'

$anypointEnvs = $anypointEnvsJson | ConvertFrom-Json
foreach ($anypointEnv in $anypointEnvs) {
    $environmentName = $anypointEnv.name
    if ($anypointEnv.type -eq "sandbox" || $anypointEnv.type -eq "design") {
        $environmentType = "development"
    } elseif ($anypointEnv.type -eq "production") {
        $environmentType = "production"
    }

    $environmentCustomProperties = @{
        "organizationId" = $anypointEnv.organizationId
        "clientId" = $anypointEnv.clientId
    }

    $environmentCustomPropertiesJson = ConvertTo-Json -InputObject $environmentCustomProperties -Depth 10 -Compress
    Write-Output "👉🏻 Creating environment $environmentName"
    az apic environment create -g $resourceGroupName -s $apicResourceName --environment-id $environmentName --title $environmentName `
            --description $environmentName" environment" --type $environmentType --custom-properties $environmentCustomPropertiesJson.replace('"','\"') `
            --server '{\"type\":\"MuleSoft API Management\"}'

    # comment one of the following lines to use the real MuleSoft data or the local file
    $anypointAPIListJson = anypoint-cli-v4 api-mgr:api:list --environment=$environmentName --output=json
    # $anypointAPIListJson = Get-Content -Raw 'api-mgr_api_list.json'

    $anypointAPIList = $anypointAPIListJson | ConvertFrom-Json
    foreach ($anypointAPI in $anypointAPIList) {
        
        $apiCustomProperties = @{
            "technology" = $anypointAPI.technology
            "stage" = $anypointAPI.stage
            "isPublic" = $anypointAPI.isPublic
        }

        $apiId = $anypointAPI.id
        $apiName = $anypointAPI.assetId
        $apiVersionId = $anypointAPI.assetVersion.replace(".","-")
        $apiVersionName = $anypointAPI.assetVersion
        $definitionId = "openapi"

        $apiCustomPropertiesJson = ConvertTo-Json -InputObject $apiCustomProperties -Depth 10 -Compress
        write-output "👉🏻 Creating API $apiName"
        az apic api create -g $resourceGroupName -s $apicResourceName --api-id $apiId --type REST `
            --title $apiName  `
            --custom-properties $apiCustomPropertiesJson.replace('"','\"')

        write-output "👉🏻 Creating version $apiVersion of API $apiName"
        az apic api version create -g $resourceGroupName -s $apicResourceName --api-id $apiId --version-id $apiVersionId  `
                    --title $apiVersionName --lifecycle-stage "production"

        write-output "👉🏻 Creating definition for version $apiVersion of API $apiName"
        az apic api definition create -g $resourceGroupName -s $apicResourceName --api-id $apiId `
            --version-id $apiVersionId --definition-id $definitionId --title "OpenAPI" --description "OpenAPI spec"      

        # uncomment the following line to use the real MuleSoft data or the local file
        anypoint-cli-v4 api-mgr:api:download-proxy --environment=Sandbox $apiId ".downloads"

        $zipFilePath = ".downloads/" + $apiName + "-" + $anypointAPI.productVersion + "-" + $anypointAPI.productVersion + "_" + $apiId + ".jar"
        if (Test-Path $zipFilePath) {
            $destinationPath = ".downloads/" + $apiId
            Expand-Archive -Path $zipFilePath -DestinationPath $destinationPath -Force
            $exchangeFilePath = ".downloads/" + $apiId + "/com/mulesoft/anypoint/gw/exchange.json"
            $exchangeJson = Get-Content -Raw $exchangeFilePath
            $exchange = $exchangeJson | ConvertFrom-Json
            $specFilePath = ".downloads/" + $apiId + "/com/mulesoft/anypoint/gw/" + $exchange.main

            write-output "👉🏻 Importing specification to the API $apiName"
            az apic api definition import-specification -g $resourceGroupName -s $apicResourceName `
                    --api-id $apiId --version-id $apiVersionId --definition-id $definitionId `
                    --specification '{\"name\":\"openapi\",\"version\":\"3.0.3\"}' `
                    --file-name $specFilePath
        }

        $deploymentId = $anypointAPI.deployment.applicationId

        # comment one of the following lines to use the real MuleSoft data or the local file
        $anypointAppJson = anypoint-cli-v4 runtime-mgr:application:describe $deploymentId --environment=$environmentName --output=json
        # $anypointAppJson = Get-Content -Raw 'runtime-mgr-application_describe.jsonn'

        $anypointApp = $anypointAppJson | ConvertFrom-Json

        $deploymentCustomProperties = @{
            "anypointAppName" = $anypointApp.name
            "targetId" = $anypointApp.target.targetId
            "runtimeVersion" = $anypointApp.target.deploymentSettings.runtimeVersion
        }

        $serverProperty = '{\"runtimeUri\":[\"' + $anypointApp.target.deploymentSettings.http.inbound.publicUrl + '\"]}'
        
        $deploymentCustomPropertiesJson = ConvertTo-Json -InputObject $deploymentCustomProperties -Depth 10 -Compress
        write-output "👉🏻 Creating deployment for the API $apiName"
        az apic api deployment create -g $resourceGroupName -s $apicResourceName --deployment-id $deploymentId `
                --title "Production deployment" --description "Petstore production deployment." --api-id $apiId `
                --environment-id "/workspaces/default/environments/$environmentName" `
                --definition-id "/workspaces/default/apis/another-petstore/versions/$apiVersionId/definitions/$definitionId" `
                --server $serverProperty `
                --custom-properties $deploymentCustomPropertiesJson.replace('"','\"')

    }
}

### 5️⃣ Discover the APIs that were just imported

You can discover the APIs with fhe following methods:
- With the Azure Portal
- With the [self-hosted API Center Portal](https://learn.microsoft.com/en-us/azure/api-center/enable-api-center-portal)
- With the [VS Code extension](https://learn.microsoft.com/en-us/azure/api-center/use-vscode-extension-copilot) that is integrated with GitHub Copilot Chat.
- With the CLI, the service REST API and more

Here we will use the [list command](https://learn.microsoft.com/en-us/cli/azure/apic/api?view=azure-cli-latest#az-apic-api-list) to display the APIs that we have just imported. 


In [ ]:
az apic api list -g $resourceGroupName -s $apicResourceName --query "[?title == 'petstore'].{Name:name, Title:title, Kind:kind, ContactEmail:contacts[0].email}" -o table

### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.